In [2]:
# Export Vault Variables 

export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"

In [3]:
## Setup entities and a group

## Enable the userpass auth method.

vault auth enable userpass

Success! Enabled userpass auth method at: userpass/


In [4]:
## Create a new user, bob with password, "training".

vault write auth/userpass/users/bob password="training"


Success! Data written to: auth/userpass/users/bob


In [5]:
## Create a new user, ellen with password, "training".

vault write auth/userpass/users/ellen password="training"


Success! Data written to: auth/userpass/users/ellen


In [6]:
## Retrieve the userpass mount accessor and save it in a file named accessor.txt.

vault auth list -format=json | jq -r '.["userpass/"].accessor' > accessor.txt

In [7]:
## Create Bob Smith entity and save the identity ID in the entity_id_bob.txt.

vault write -format=json identity/entity name="Bob Smith" \
        policies="read-cred" \
        metadata=team="Processor" \
        | jq -r ".data.id" > entity_id_bob.txt

In [8]:
## Add an entity alias for the Bob Smith entity.

vault write identity/entity-alias name="bob" \
      canonical_id=$(cat entity_id_bob.txt) \
      mount_accessor=$(cat accessor.txt)


Key             Value
---             -----
canonical_id    a0486d55-1b91-28fa-f63c-650b87e590c1
id              b8bc4cfc-a818-f79a-576b-6a72b0302dd6


In [9]:
## Create Ellen Wright entity and save the identity ID in the entity_id_ellen.txt

vault write -format=json identity/entity name="Ellen Wright" \
        policies="default" \
        metadata=team="Acct Controller" \
        | jq -r ".data.id" > entity_id_ellen.txt


In [10]:
## Add an entity alias for the Ellen Wright entity.

vault write identity/entity-alias name="ellen" \
      canonical_id=$(cat entity_id_ellen.txt) \
      mount_accessor=$(cat accessor.txt)


Key             Value
---             -----
canonical_id    8aafa539-854f-22e4-0547-2c45551d52be
id              4e39e09b-120e-5a49-3642-b9f9df236a0c


In [11]:
## Create acct_manager group and add Ellen Wright entity as a member.

vault write identity/group name="acct_manager" \
      policies="acct_manager" \
      member_entity_ids=$(cat entity_id_ellen.txt)


Key     Value
---     -----
id      94c04988-c41d-3b0e-d419-f331aeddac1f
name    acct_manager
